## Imports

In [49]:
import ed25519
import x25519
from aes import aes
import aes
import base64
import socket
import os

## Full apllication

In [97]:
# const
adress = '192.168.127.121'    
port = 20582        

# ed25519 private Key
priv_ed25519 = ed25519.keys.SigningKey(base64.b64decode("06DAozc2xb72sdoseInPHTLwhg+E3+pC+qPhxCh12To="))
print(f"Public Key to save in arduino :{base64.b64encode(priv_ed25519.get_verifying_key().to_bytes())}")

Public Key to save in arduino :b'GskUXMWVAz26Ko+AAywcB75HGZurjfnznrgsxaSY1mc='


In [100]:

# TCP server
s = socket.socket()           
s.bind((adress, port))         
print ("socket binded to %s" %(port))
s.listen(5)    
print ("socket is listening")    

# connect first client
c, addr = s.accept()    
print (f"Got connection from {addr} \n" )
c.settimeout(5)

# receive public ed25519 key from server then accept
# Normally fisrt time add to known host
client_pub_ed25519 = ed25519.keys.VerifyingKey(base64.b64decode(c.recv(44).decode("ascii"))) 
c.send('accept'.encode("ascii"))
print(f"client with \n{client_pub_ed25519.to_bytes().hex()}\npublic key is accepted\n")

# get public x25519 from client and control with signature
# control signature will be usefull if public ed25519 can be verified with known host
base64_client_pub_x25519 = c.recv(44).decode("ascii")
client_pub_x25519 = base64.b64decode(base64_client_pub_x25519)
client_signature = c.recv(64)

# control signature 
try:
    client_pub_ed25519.verify(client_signature.hex(), base64_client_pub_x25519.encode("ascii"), encoding='hex')
    print("The signature is valid.")
except:
    print("Invalid signature!")

# authenticate us
priv_x25519 =  os.urandom(32)
pub_x25519 = x25519.scalar_base_mult(priv_x25519)
encoded_pub_x25519 = base64.b64encode(pub_x25519)

signature = priv_ed25519.sign(encoded_pub_x25519)
c.send(encoded_pub_x25519)
c.send(signature)

shared_from_client = c.recv(32)
print(f"Shared from client  : {(shared_from_client).hex()}")

calculate_shared = x25519.curve25519(priv_x25519, client_pub_x25519)

print(f"Shared              : {(calculate_shared).hex()}")


s.close()

socket binded to 20582
socket is listening
Got connection from ('192.168.127.97', 50103) 

client with 
698d81378841e26489372328c620abbb778d96388e7913ca478374e6ea95a5ad
public key is accepted

The signature is valid.
Shared from client  : f01170746251d06edb02ab084fad29170a0f7fb4eb151197ab9a18012564d708
Shared             : f01170746251d06edb02ab084fad29170a0f7fb4eb151197ab9a18012564d708


## Individual cells for testing

### Signatures tests

In [2]:
privKey = ed25519.keys.SigningKey(b'0FDE8A96B2066E34FBEAC155FC5104A8C05A96C501A001300EE99F9EB261D82D',encoding='hex')
pubKey = ed25519.keys.VerifyingKey(b'7E4A530129FBE298F9E431356F0D2F5687FC21A1241064EA224687DE8BE5A24A',encoding='hex')

print("Private key (32 bytes):", privKey.to_seed().hex())
print("Public key (32 bytes): ", pubKey.to_bytes().hex())


msg = "test de signature".encode("ascii")

# signature = privKey.sign(msg, encoding='hex')
signature = b'B785B9A12C6389EDF447D470D2B38C278E5ADBC31F67FA79741B9AC8CA78FB9EB96585DE2DFD2ABCF4D769163F07AEBC412424247824EE9BF2F087FA7F5FD10B'

print("Signature (64 bytes):", signature)
try:
    pubKey.verify(signature, msg, encoding='hex')
    print("The signature is valid.")
except:
    print("Invalid signature!")


Private key (32 bytes): 0fde8a96b2066e34fbeac155fc5104a8c05a96c501a001300ee99f9eb261d82d
Public key (32 bytes):  7e4a530129fbe298f9e431356f0d2f5687fc21a1241064ea224687de8be5a24a
Signature (64 bytes): b'B785B9A12C6389EDF447D470D2B38C278E5ADBC31F67FA79741B9AC8CA78FB9EB96585DE2DFD2ABCF4D769163F07AEBC412424247824EE9BF2F087FA7F5FD10B'
The signature is valid.


In [95]:
pubKey = ed25519.keys.VerifyingKey(base64.b64decode("vRy71JXxwpbl3ZfCJCeROrUrYbxD8DMuu0qsCTIa1Oc=").hex(),encoding='hex')

msg = "hYSGOw1znznl5NO3OnI85/1kiC75dUEH0u1BZLDFLWw=".encode("ascii")

signature = b'0f2f532b651886ad13022057cf3c2acc7daf573ed48487484d62d5s4c972354de7390ff041561dfbe35877c0575a199ade4200d2ddf5efb92e9f57ad12b542102'

try:
    pubKey.verify(signature, msg, encoding='hex')
    print("The signature is valid.")
except:
    print("Invalid signature!")

Invalid signature!


### Create random keys 

In [90]:
# privKey2, pubKey2 = ed25519.keys.create_keypair()
privKey2 = ed25519.keys.SigningKey(base64.b64decode("06DAozc2xb72sdoseInPHTLwhg+E3+pC+qPhxCh12To="))
pubKey2 = privKey2.get_verifying_key()
print("Private key (32 bytes):", privKey2.to_seed().hex())
print("Public key (32 bytes): ", pubKey2.to_bytes().hex())

Private key (32 bytes): d3a0c0a33736c5bef6b1da2c7889cf1d32f0860f84dfea42faa3e1c42875d93a
Public key (32 bytes):  1ac9145cc595033dba2a8f80032c1c07be47199bab8df9f39eb82cc5a498d667


### Test Curve25519

In [5]:
print("arduino Private key (32 bytes):", privKey.to_seed().hex())
print("arduino Public key (32 bytes): ", pubKey.to_bytes().hex())

print("python Private key (32 bytes):", privKey2.to_seed().hex())
print("python Public key (32 bytes): ", pubKey2.to_bytes().hex())

pubx = x25519.scalar_base_mult(privKey.to_seed())
print(pubx.hex())
pubx2 = x25519.scalar_base_mult(privKey2.to_seed())
print(pubx2.hex())

shared = x25519.curve25519(privKey.to_seed(), pubx2) 
sharedPy = x25519.curve25519(privKey2.to_seed(), pubx) 

print (f"\n\nShared (arduino):\t{(shared).hex()}")
print (f"Shared (python): \t{(sharedPy).hex()}")

arduino Private key (32 bytes): 0fde8a96b2066e34fbeac155fc5104a8c05a96c501a001300ee99f9eb261d82d
arduino Public key (32 bytes):  e7432ca9d9028065e54bfb9d05b8eaadaff6f4924ea62db25e17c24a71048d20
python Private key (32 bytes): cb52809cdf1d8bc9baeb58e2dacacb77a47bba67260b107b55821a991a113bfe
python Public key (32 bytes):  96083a0efa3eea15bc5746e8aa257e7082a3a186240af52ecf40c4c13dfe052c
55af02898e27b564b5701805120b21b57d2f3d2fded8bd9717c566f7adaace33
56ac5c5c7ea098ecf6a3c747ffe3acd3016aaf28cfa408ce79bcbef21138dc3d


Shared (arduino):	49049308d1922bf3fb3c6599187260bcac7523f4d36b9de8d93a87593e00400a
Shared (python): 	49049308d1922bf3fb3c6599187260bcac7523f4d36b9de8d93a87593e00400a


### Save sever public key in arduino (Base64)

In [61]:
print(f"Public Key to save in arduino :{base64.b64encode(pubKey2.to_bytes())}")

Public Key to save in arduino :b'06DAozc2xb72sdoseInPHTLwhg+E3+pC+qPhxCh12To='
Public Key to save in arduino :b'GskUXMWVAz26Ko+AAywcB75HGZurjfnznrgsxaSY1mc='


In [91]:
privx2 =  os.urandom(32)
pubx2 = x25519.scalar_base_mult(privx2)

encoded = base64.b64encode(pubx2)

print(f"Key in hex  :{pubx2.hex()}")
print(f"Key to send :{encoded}")
 
signatureofencoded = privKey2.sign(encoded)
print(f"Signature :{signatureofencoded.hex()}")

Key in hex  :20f18bd2680f7dc245d4094ed90e823cae997f69bc828f97ee1d8f1a2eb0ed25
Key to send :b'IPGL0mgPfcJF1AlO2Q6CPK6Zf2m8go+X7h2PGi6w7SU='
Signature :cc029c64093e71460c57946138c82caa6a644b1da656a6e9c462d2e846fc32a8d718862f698b79c289b6d438f262a883d60266c9187101416166ca687bc7460c


In [81]:
try:
    pubKey2.verify(signatureofencoded.hex(), encoded, encoding='hex')
    print("The signature is valid.")
except:
    print("Invalid signature!")

The signature is valid.


### From public x25519 key of arduino create shared key

In [59]:
arduinoKey = base64.b64decode("WkS78GGKoSTRUB6/MOKcixwXDz9Xl0expuPZ5sES9C4=")
print(arduinoKey.hex())

realshared = x25519.curve25519(privKey2.to_seed(), arduinoKey)

print (f"Shared : \t{(realshared).hex()}")

5a44bbf0618aa124d1501ebf30e29c8b1c170f3f579747b1a6e3d9e6c112f42e
Shared : 	aff041dea1c51a458a5d3aa15027c191c9dd6f9f17f1fac97e3a720de383541c


### TCP server 

In [92]:
s = socket.socket()        
print ("Socket successfully created")

port = 20582               

s.bind(('192.168.127.121', port))        

print ("socket binded to %s" %(port))
 
s.listen(5)    
print ("socket is listening")         

Socket successfully created
socket binded to 20582
socket is listening


In [62]:
s.close()

### Accept

In [93]:
c, addr = s.accept()    
print ('Got connection from', addr )

c.settimeout(5)

Got connection from ('192.168.127.97', 54298)


### Send end receive to control agreement

In [94]:
print (c.recv(44).decode("ascii"))
c.send('accept'.encode("ascii"))

print (c.recv(44).decode("ascii"))
print (c.recv(64).hex())

c.send(encoded)
c.send(signatureofencoded)

print (c.recv(64).hex()) 
print (c.recv(32).decode("ascii")) 
print (c.recv(32).hex()) 

#   c.close()


D/+PPH9QVqDSijJcxFIjVEAaEsWKG/C4VLLYDor7R2Y=
K1oVXq2YOAuVoXvjVpdiq+s732cqRAtZwvqlBEVVg1o=
61b5a916bb94b0861d66f10e57c251818f61fe39a5da462f263b1af95d18dfa5ef5e7b10d56323513a25af92d56377723395ed03c05f3849ca86dfadfb9a2f06
596f75206172652061206c696172


timeout: timed out

In [31]:
data = bytes()
try:
    print (c.recv(44).decode("ascii"))
    c.settimeout(0.01)
    while True:
        data +=c.recv(1)
except:
    print(data)

b''


### Tests AES encrypting

In [11]:
pt = 0x00112233445566778899aabbccddeeff

mk =  int.from_bytes(realshared,'big')

cipher = aes.aes(mk, 256)
print(mk)

ct = cipher.enc_once(pt)
print(ct)
print("0x"+hex(aes.utils.arr8bit2int(ct))[2:].zfill(32))

pr = cipher.dec_once(ct)
print(pr)
print("0x"+hex(aes.utils.arr8bit2int(pr))[2:].zfill(32))

31452682052524923878346587304201317347138921881689638911249933624929528380428
[3, 236, 22, 99, 205, 24, 79, 51, 229, 61, 172, 239, 115, 34, 49, 76]
0x03ec1663cd184f33e53dacef7322314c
[0, 17, 34, 51, 68, 85, 102, 119, 136, 153, 170, 187, 204, 221, 238, 255]
0x00112233445566778899aabbccddeeff
